In [ ]:

!pip install pillow_heif

import cv2,numpy as np,matplotlib.pyplot as plt
from google.colab import files
from IPython.display import Image,display
import pillow_heif
from PIL import Image as PILImage
import os

uploaded=files.upload()
filename=next(iter(uploaded))

if filename.lower().endswith('.heic'):
    heif_file=pillow_heif.read_heif(filename)
    image=PILImage.frombytes(heif_file.mode,heif_file.size,heif_file.data,"raw",heif_file.mode,heif_file.stride)
    image.save('temp.jpg','JPEG')
    image=cv2.imread('temp.jpg')
    os.remove('temp.jpg')
else:
    image=cv2.imread(filename)

image_rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.imshow(image_rgb)
plt.title('Input')
plt.axis('off')


In [ ]:
params={'hsv_lower':np.array([0,0,180]),'hsv_upper':np.array([180,60,255]),'min_area':5000,'max_area':100000}

def detect(image,params):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,params['hsv_lower'],params['hsv_upper'])
    
    kernel=np.ones((11,11),np.uint8)
    mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
    mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    
    contours,_=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    result=image.copy()
    objects=[]
    
    for c in contours:
        area=cv2.contourArea(c)
        if params['min_area']<area<params['max_area']:
            perimeter=cv2.arcLength(c,True)
            circularity=4*np.pi*area/(perimeter*perimeter) if perimeter>0 else 0
            
            (x,y),radius=cv2.minEnclosingCircle(c)
            center=(int(x),int(y))
            radius=int(radius)
            
            aspect_ratio=1
            if len(c) > 5:
                (x,y),(MA,ma),angle=cv2.fitEllipse(c)
                aspect_ratio=ma/MA if MA > 0 else 1
            
            is_ball=circularity>0.6 and 0.8<aspect_ratio<1.2
            label="Ball" if is_ball else "Paper"
            color=(0,255,0) if is_ball else (255,0,0)
            
            if is_ball:
                cv2.circle(result,center,radius,color,3)
                cv2.putText(result,"Ball",center,cv2.FONT_HERSHEY_SIMPLEX,1.5,color,3)
            
            objects.append((int(x),int(y),label))
    
    return result,mask,objects

result,mask,objects=detect(image,params)
result_rgb=cv2.cvtColor(result,cv2.COLOR_BGR2RGB)

plt.subplot(122)
plt.imshow(result_rgb)
plt.title(f'Objects: {len(objects)}')
plt.axis('off')
plt.tight_layout()
plt.show()

plt.figure(figsize=(5,5))
plt.imshow(mask,cmap='gray')
plt.title('Mask')
plt.axis('off')
plt.show()

print("\nDetected Objects:")
for i,(x,y,label) in enumerate(objects,1):
    print(f"{i}. {label}")


# Need to adjust the detection? Here's how! 🔧

If the detector isn't finding the white balls correctly, you can adjust these settings in the `white_detector` dictionary:

## If the wrong color is being detected:
Change these numbers in `white_detector`:
```python
'hsv_lower': np.array([0, 0, 200])   
'hsv_upper': np.array([180, 30, 255]) 
```

## If wrong sized objects are being detected:
Change these numbers:
```python
'min_area': 100 
'max_area': 5000 
```

Just change the numbers and run the cell again to see what happens! 🔄
Try small changes first - for example, change 200 to 180 or 220.


# Object Detection with OpenCV
This notebook demonstrates:
1. AprilTag detection using the `apriltag` library
2. White ball detection using HSV color thresholding
3. Visualization of detected objects

First, let's install the required packages:
